<a href="https://colab.research.google.com/github/andriasmagno/andriasmagno-YouTube-Insights-Extractor/blob/main/Puchando_dados_da_Api_do_Youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from googleapiclient.discovery import build
import pandas as pd

# Chave da API que você obteve
api_key = 'sua chave aqui'

# Função para obter o ID do canal a partir do nome personalizado
def get_channel_id_by_custom_url(custom_url):
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Buscar o canal pelo nome personalizado
    search_request = youtube.search().list(
        part='snippet',
        q=custom_url,
        type='channel',
        maxResults=1
    )
    search_response = search_request.execute()

    # Verificar se há resultados e extrair o ID do canal
    if 'items' in search_response and len(search_response['items']) > 0:
        return search_response['items'][0]['snippet']['channelId']
    else:
        print("Nenhum canal encontrado com o nome fornecido.")
        return None

# Função para obter vídeos a partir do ID do canal
def get_channel_videos(channel_id):
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Puxar informações do canal
    channel_request = youtube.channels().list(
        part='contentDetails',
        id=channel_id
    )
    channel_response = channel_request.execute()

    if 'items' not in channel_response or len(channel_response['items']) == 0:
        print("Nenhum item encontrado para o ID do canal fornecido.")
        return pd.DataFrame()

    # Extrair o playlist ID dos vídeos enviados pelo canal
    playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    # Listar vídeos no canal
    video_ids = []
    next_page_token = None

    while True:
        playlist_request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=10,  # Ajuste o limite conforme necessário
            pageToken=next_page_token
        )
        playlist_response = playlist_request.execute()

        for item in playlist_response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = playlist_response.get('nextPageToken')
        if next_page_token is None:
            break

    # Puxar detalhes dos vídeos
    videos = []
    for video_id in video_ids:
        video_request = youtube.videos().list(
            part='snippet,statistics',
            id=video_id
        )
        video_response = video_request.execute()

        for item in video_response['items']:
            video_data = {
                'title': item['snippet']['title'],
                'published_at': item['snippet']['publishedAt'],
                'views': item['statistics'].get('viewCount', 0),
                'likes': item['statistics'].get('likeCount', 0),
                'comments': item['statistics'].get('commentCount', 0)
            }
            videos.append(video_data)

    return pd.DataFrame(videos)

# Nome personalizado do canal
custom_url = 'nome do canal'

# Buscar o ID do canal pelo nome personalizado
channel_id = get_channel_id_by_custom_url(custom_url)

# Se o ID do canal for encontrado, puxar os vídeos
if channel_id:
    df_videos = get_channel_videos(channel_id)

    # Exibir os dados, se houver vídeos
    if not df_videos.empty:
        print(df_videos.head())
    else:
        print("Nenhum vídeo encontrado para o canal.")
